# Carga de datos

**Librerías**:

In [1]:
!pip install pyjanitor==0.22.0

In [2]:
import janitor
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno
import nhanes.load

In [3]:
%run pandas-missing-extension.ipynb

## Descripción

- Documentación del DataSet: https://wwwn.cdc.gov/nchs/nhanes/tutorials/default.aspx

**Pasos para procesar y limpiar datos:**
- Selección de variables con las que se trabajaran.
- Cambio de nombres de variables a nombres cortes y prolijos.
- Estandarización de valores faltnates.
- Ordenar dataset de mayor a menor cantidad de valores faltantes por variable.
- Eliminar observaciones con valores faltantes: observaciones de valores faltantes en una variable cuando sea una proporción no significativa o muy pequeña.
- Estandarización de tipo de datos


## Carga de datos `NHANES`

In [4]:
nhanes_raw_df = (
  nhanes.load.load_NHANES_data(year="2017-2018") # Carga los datos del 2017 y 2018
  .clean_names(case_type="snake") # Estandatiza nombre de variables a un tipo snake_case
)
print(nhanes_raw_df.shape)

(8366, 197)


## Procesar y limpiar los datos `NHANES`

In [5]:
(
  nhanes_raw_df
  .select_columns( # 1. Seleccion de variables
    "general_health_condition",
    "age_in_years_at_screening",
    "gender",
    "current_selfreported_height_inches",
    "current_selfreported_weight_pounds",
    "doctor_told_you_have_diabetes",
    "60_sec_pulse30_sec_pulse2",
    "total_cholesterol_mgdl"
  )
  .rename_columns({ # 2. Cambio de nombre de variables
    "general_health_condition": "health_condition",
    "age_in_years_at_screening": "years",
    "current_selfreported_height_inches": "height",
    "current_selfreported_weight_pounds": "weight",
    "doctor_told_you_have_diabetes": "diabetes",
    "60_sec_pulse30_sec_pulse2": "pulse",
    "total_cholesterol_mgdl": "cholesterol"
  })
  .replace({
    "height": {
      999: np.nan,
      777: np.nan
    },
    "weight": {
      999: np.nan,
      777: np.nan
    },
    "diabetes": {
      "Borderline": np.nan
    }
  })
  .missing.sort_variables_by_missingness() # Ordenamos datos
  .dropna( # Eliminamos los valores faltantes con baja proporción
    subset=["diabetes"],
    how = "any"
  )
  .transform_column( # Estandarización del tipo de dato
    column_name="diabetes",
    function = lambda s: s.astype(int),
    elementwise = False
  )
)

,height,weight,health_condition,cholesterol,pulse,diabetes,years,gender
SEQN,,,,,,,,
93703.0,NaN,NaN,NaN,NaN,NaN,0,2.0,Female
93704.0,NaN,NaN,NaN,NaN,NaN,0,2.0,Male
93705.0,63.0,165.0,Good,157.0,52.0,0,66.0,Female
93706.0,68.0,145.0,Very good,148.0,82.0,0,18.0,Male
93707.0,NaN,NaN,Good,189.0,100.0,0,13.0,Male
...,...,...,...,...,...,...,...,...
102951.0,NaN,NaN,NaN,NaN,NaN,0,4.0,Male
102953.0,65.0,218.0,Fair or,182.0,78.0,0,42.0,Male
102954.0,66.0,150.0,Good,172.0,78.0,0,41.0,Female
